# TP2
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 1

### Descrição do Problema

É nos dado um Control Flow Automaton (CFA) que descreve um programa imperativo cujo objetivo é implementar a multiplicação de dois inteiros a,b, fornecidos como "input" e um n, também fornecido como "input", de precisão limitada.
Para alem disso, temos de ter em conta os seguintes aspetos:
- Existe a possibilidade de alguma das operações do programa produzir um erro de “overflow”;
- Os nós do grafo representam ações  que actuam sobre os “inputs” do nó e produzem um “output” com as operações indicadas;
- Os ramos do grafo representam ligações que transferem o “output” de um nodo para o “input” do nodo seguinte. Esta transferência é condicionada pela satisfação da condição associada ao ramo.

### Abordagem do Problema

Para resolver este problema, vamos construir um First Order Transition System (FOTS) usando um BitVector de tamanho n de forma a descrever o comportamento do autómato acima mencionado.

( Talvez seja fixe mencionar a escolha do solver aqui futuramente)

Para além do FOTS, também vamos verificar se $P≡(x∗y+z=a∗b)$ é um invariante do comportamento que estamos a estudar.


## Código Python

variaveis -> x,y,z e o program counter (nº de linhas de código)

x,y,z = a,b,0

ideia 1:
0: while(y!=0):

1: 	if even(y) then x,y,z = 2*x,y/2,z

2:	if odd(y)  then x,y,z = x,y-1,z+x

3: stop

ideia 2:

0: while(y!=0):

 	if even(y) then x,y,z = 2*x,y/2,z

	if odd(y)  then x,y,z = x,y-1,z+x

1: stop


outras ideias que podem ser importantes: na aula T do dia 4 vimos que o stor evita usar o diferente por causa de ter problemas com indicibilidade, devemos também fazer o mesmo, vou cometer o erro aqui em baixo de usar not equals mas talvez temos de trocar depois.não esquecer

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import random as rn

# vou declarar aqui um exemplo depois passamos para exemplos separados
n = 32 # [0,0] [0,1] [1,0] [1,1]
a = rn.randrange(1, 2**(n-16))
b = rn.randrange(1, 2**(n-16))
k = 2*n
k_inv = 1
def toInt(s):
    return sum([int(x)*2**(len(s)-i-1) for i,x in (enumerate(s))])

print('x = ', BV(a,n).bv_str())
print('y = ', BV(b,n).bv_str())

x =  00000000000000001111011010111001
y =  00000000000000000001111100000010


 $$\mathsf{trans}(x,y,z,pc,x',y',z',pc')\;\equiv\;$$
 $$\left\{\begin{array}{lr} (y=0)\land (pc=0) \land (x'=x)\land (y'=y) \land (c'=c) \land (pc'=3) & \lor \\ (a > b)\land(c=0)\land (a'+b=a) \land (b'=b)\land (c'=0) & \lor \\ (a < b)\land (c=0) \land (b'+a = b)\land (a'=a)\land (c'=0) & \lor \\ (c=1)\land(c'=1)\land (a'=a)\land (b'=b) & \end{array}\right.$$


In [2]:
# Ideias à toa, não faz muito sentido o que está escrito aqui a baixo

def BVFirst(n):
    return BV(2**(n-1),n)

def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),types.BVType(n+1))
    state['y'] = Symbol('y'+str(i),types.BVType(n+1))
    state['z'] = Symbol('z'+str(i),types.BVType(n+1))
    return state

def init(state):
    tPc = Equals(state['pc'],Int(0)) # Program counter a zero
    tZ = Equals(state['z'],BVZero(n+1)) # Z a zero
    tX = Equals(state['x'], BV(a, n+1)) # x inicilizado com valor de a 2**(n-16)
    tY = Equals(state['y'], BV(b,n+1)) # y inicilizado com valor de b
    return And(tPc,tX,tY,tZ)

def tStop(curr,prox):
    tPcZero = Equals(curr['pc'],Int(0))
    tYZero = Equals(curr['y'],BVZero(n+1))#y=0
    tZFirst = Equals(BVAnd(curr['z'],BVFirst(n+1)),BVZero(n+1))#primriro bit de z = 0
    tX = Equals(prox['x'],curr['x'])
    tY = Equals(prox['y'],curr['y'])
    tZ = Equals(prox['z'],curr['z'])
    tPc = Equals(prox['pc'],Int(1))
    return And(tYZero,tZFirst,tPcZero,tX,tY,tZ,tPc)

def tEven(curr,prox):
    tPcZero = Equals(curr['pc'],Int(0)) #dam este e o stop sao ambiguos
    tYLast = Equals(BVAnd(curr['y'],BVOne(n+1)),BVZero(n+1))#ultimo bit = 0
    tYGt = BVUGT(curr['y'],BVZero(n+1))#y > 0
    tX = Equals(prox['x'], BVLShl(curr['x'],BVOne(n+1)))#2*x
    tY = Equals(prox['y'], BVLShr(curr['y'],BVOne(n+1)))#y/2
    tZ = Equals(prox['z'],curr['z'])#z
    tPc = Equals(prox['pc'],Int(0))
    return And(tPcZero,tYLast,tYGt,tX,tY,tZ,tPc)

def tError(curr,prox):
    tPcZero = Equals(curr['pc'],Int(0))
    tYFirst = Equals(BVAnd(curr['y'],BVFirst(n+1)),BVFirst(n+1))
    tXFirst = Equals(BVAnd(curr['x'],BVFirst(n+1)),BVFirst(n+1))
    tZFirst = Equals(BVAnd(curr['z'],BVFirst(n+1)),BVFirst(n+1))
    tX = Equals(prox['x'], curr['x'])
    tY = Equals(prox['y'],curr['y'])
    tZ = Equals(prox['z'],curr['z'])
    tPc = Equals(prox['pc'],Int(2))
    return And(tPcZero,Or(tYFirst,tXFirst,tZFirst),tX,tY,tZ,tPc)

def tEndError(curr,prox):
    tPcTwo = Equals(curr['pc'],Int(2))
    tX = Equals(prox['x'], curr['x'])
    tY = Equals(prox['y'],curr['y'])
    tZ = Equals(prox['z'],curr['z'])
    tPc = Equals(prox['pc'],Int(2))
    return And(tPcTwo,tX,tY,tZ,tPc)

def tOdd(curr,prox):
    tPcZero = Equals(curr['pc'],Int(0))
    tYLast = Equals(BVAnd(curr['y'],BVOne(n+1)),BVOne(n+1))
    tX = Equals(prox['x'], curr['x'])
    tY = Equals(prox['y'],BVSub(curr['y'],BVOne(n+1)))
    tZ = Equals(prox['z'],BVAdd(curr['x'],curr['z'])) 
    tPc = Equals(prox['pc'],Int(0))   
    return And(tPcZero,tYLast,tX,tY,tZ,tPc)

def tEnd(curr,prox):
    tPcOne = Equals(curr['pc'],Int(1))
    tX = Equals(prox['x'],curr['x'])
    tY = Equals(prox['y'],curr['y'])
    tZ = Equals(prox['z'],curr['z'])
    tPc = Equals(prox['pc'],Int(1))
    return And(tPcOne,tX,tY,tZ,tPc)

def trans(curr,prox):
    tToStop = tStop(curr,prox)
    tToEven   = tEven(curr,prox)
    tToError  = tError(curr,prox)
    tToEndError = tEndError(curr,prox)
    tToOdd    = tOdd(curr,prox)
    tToEnd    = tEnd(curr,prox)
    return Or(tToStop,tToEven,tToError,tToEndError,tToOdd,tToEnd)



In [3]:
with Solver(name="msat") as s:
    
        # completar
    
        # cria k copias do estado
        trace = [declare(i) for i in range(k)]
        #print(trace)
        # criar o traço
        s.add_assertion(init(trace[0]))
        #print(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
        
        if s.solve():
            for i in range(k):
                print(i)
                print("pc=", pc := s.get_value(trace[i]['pc']).constant_value())
                print("x=", toInt(s.get_value(trace[i]['x']).bv_str()))
                print("y=", toInt(s.get_value(trace[i]['y']).bv_str()))
                print("z=", toInt(s.get_value(trace[i]['z']).bv_str()))
                print()
                if pc in (1,2):
                    break
        else:
            print('no')

0
pc= 0
x= 63161
y= 7938
z= 0

1
pc= 0
x= 126322
y= 3969
z= 0

2
pc= 0
x= 126322
y= 3968
z= 126322

3
pc= 0
x= 252644
y= 1984
z= 126322

4
pc= 0
x= 505288
y= 992
z= 126322

5
pc= 0
x= 1010576
y= 496
z= 126322

6
pc= 0
x= 2021152
y= 248
z= 126322

7
pc= 0
x= 4042304
y= 124
z= 126322

8
pc= 0
x= 8084608
y= 62
z= 126322

9
pc= 0
x= 16169216
y= 31
z= 126322

10
pc= 0
x= 16169216
y= 30
z= 16295538

11
pc= 0
x= 32338432
y= 15
z= 16295538

12
pc= 0
x= 32338432
y= 14
z= 48633970

13
pc= 0
x= 64676864
y= 7
z= 48633970

14
pc= 0
x= 64676864
y= 6
z= 113310834

15
pc= 0
x= 129353728
y= 3
z= 113310834

16
pc= 0
x= 129353728
y= 2
z= 242664562

17
pc= 0
x= 258707456
y= 1
z= 242664562

18
pc= 0
x= 258707456
y= 0
z= 501372018

19
pc= 1
x= 258707456
y= 0
z= 501372018



Provar invariante

In [ ]:
def invariant(state):
    return Equals(BVAdd(BVMul(state['x'], state['y']), state['z']), BVMul(BV(a, n+1), BV(b, n+1)))


def kinduction_always(declare,init,trans,inv,k):
    with Solver(name="msat") as s:
        
        
        traco = [declare(i) for i in range(k+1)]
       
        s.push()
        s.add_assertion(init(traco[0]))
        
        for i in range(k-1):
            s.add_assertion(trans(traco[i],traco[i+1]))
        s.add_assertion(Or([Not(inv(traco[i])) for i in range(k)]))
        
        if s.solve():
            print('A propriedade nao e nos k estados iniciais')

            for v in traco[0]:
                print(v,'=',s.get_value(traco[0][v]))
            return
        s.pop()
        
        #testar o passo k-indutivo
        
        s.push()
        
        for i in range(k):
            s.add_assertion(trans(traco[i],traco[i+1]))
            s.add_assertion(inv(traco[i]))
        s.add_assertion(Not(inv(traco[k])))    
        
        
        if s.solve():
            print('O passo %s indutivo nao preserva a propriedade' % k)
            print('O ponto onde falha começa aqui:')

            for v in traco[0]:
                print(v,'=',s.get_value(traco[0][v]))
            return
        s.pop()
        
        print("A prop verifica-se")
        
        


In [ ]:
kinduction_always(declare,init,trans,invariant,k_inv)